In [ ]:
import pandas as pd

# Load files
loyalty_rules = pd.read_csv("/content/loyalty_rules.csv")
sales = pd.read_csv("/content/store_sales_header (1).csv")
customers = pd.read_csv("/content/customer_details.csv")

# Base loyalty rule
base_rule = loyalty_rules[loyalty_rules["rule_name"] == "RULE_02"].iloc[0]

# Calculate base points
sales["base_points"] = sales["total_amount"] * base_rule["points_per_unit_spend"]


big_basket_bonus_points = 50  # Default bonus points for a 'BigBasket'
big_basket_min_spend_threshold = 1000  # Default minimum spend for 'BigBasket'

sales["bonus_points"] = sales["total_amount"].apply(
    lambda x: big_basket_bonus_points if x >= big_basket_min_spend_threshold else 0
)

# Total points per transaction
sales["earned_points"] = sales["base_points"] + sales["bonus_points"]

# Aggregate per customer
customer_points = sales.groupby("customer_id")["earned_points"].sum().reset_index()

# Update customer table
customers_updated = customers.merge(customer_points, on="customer_id", how="left")
customers_updated["earned_points"] = customers_updated["earned_points"].fillna(0)
customers_updated["updated_loyalty_points"] = customers_updated["total_loyalty_points"] + customers_updated["earned_points"]

customers_updated.head()

,customer_id,first_name,email,loyalty_status,total_loyalty_points,last_purchase_date,segment_id,earned_points,updated_loyalty_points
0,1,Diya,diya1@mail.com,Bronze,204,2025-01-01,9,8573.925,8777.925
1,2,Anaya,anaya2@mail.com,Bronze,4837,2025-01-05,10,9850.675,14687.675
2,3,Aarav,aarav3@mail.com,Silver,4597,2025-01-02,8,13667.410,18264.410
3,4,Aman,aman4@mail.com,Bronze,2278,2025-01-06,7,18968.700,21246.700
4,5,Rahul,rahul5@mail.com,Platinum,2276,2025-01-01,7,10869.900,13145.900


In [9]:
import pandas as pd

# Save the updated customer details to a CSV file
customers_updated.to_csv('customers_updated.csv', index=False)
print('customers_updated.csv has been saved.')

# Save the RFM analysis results to a CSV file
rfm.to_csv('rfm_analysis_results.csv', index=False)
print('rfm_analysis_results.csv has been saved.')

customers_updated.csv has been saved.
rfm_analysis_results.csv has been saved.


The `customers_updated.csv` and `rfm_analysis_results.csv` files have been saved to your Colab environment. You can find them in the file browser on the left sidebar.

In [10]:
import pandas as pd
from datetime import datetime

sales = pd.read_csv("store_sales_header (1).csv")
customers = pd.read_csv("customer_details.csv")

# Convert date column
sales["transaction_date"] = pd.to_datetime(sales["transaction_date"])

# Current date for recency calculation
current_date = sales["transaction_date"].max()

rfm = sales.groupby("customer_id").agg(
    recency_days=("transaction_date", lambda x: (current_date - x.max()).days),
    frequency=("transaction_id", "count"),
    monetary=("total_amount", "sum")
).reset_index()

rfm = rfm.merge(
    customers[["customer_id", "total_loyalty_points"]],
    on="customer_id",
    how="left"
)

rfm["total_loyalty_points"] = rfm["total_loyalty_points"].fillna(0)


def assign_segment(row):
    if row["recency_days"] <= 30 and row["frequency"] >= 5 and row["monetary"] >= 5000:
        return "High-Value"
    elif row["frequency"] >= 5 and row["total_loyalty_points"] >= 500:
        return "Loyal"
    elif row["recency_days"] > 60 and row["total_loyalty_points"] > 0:
        return "At-Risk"
    else:
        return "Occasional"

rfm["customer_segment"] = rfm.apply(assign_segment, axis=1)

rfm.head()


,customer_id,recency_days,frequency,monetary,total_loyalty_points,customer_segment
0,1,2,2,16947.85,204,Occasional
1,2,0,2,19501.35,4837,Occasional
2,3,0,3,27034.82,4597,Occasional
3,4,0,3,37637.40,2278,Occasional
4,5,0,4,21339.80,2276,Occasional
